# Small tutorial on iRODS paths

* iRODS deals with POSIX-like paths, i.e. paths use the `/` as delimiter and all absolute paths start with `/`.
* In default iRODS instances users have a personal home collection `/<zonename>/home/<username>`
* In some iRODS instances like in Yoda instances, users are part of a group and only have access to group collections `/<yodazone>/home/<groupname>`

In *iBridges* we allow all paths by default to be strings, which have the to be formatted as described above, i.e. for up and downloads all source and destination paths can be offered as `str`.
However, we also offer a small class `IrodsPath` with which some formatting is done automatically.

## The iRODS home

In your configuration file `irods_environment.json` you can set a default working directory on the iRODS instance with:
```
"irods_home": "/<zone>/home/<user or groupname>"
```
This information is stored in the iBridges session object and employed by the iBridges IrodsPath object.

### Create a session and set your iRODS default collection

In [ ]:
from ibridges.interactive import interactive_auth
from pathlib import Path

In [ ]:
session = interactive_auth()

In [ ]:
session.home

There are three ways to set the `irods_home`:

1. You can set the "irods_home" in the configuration file `irods_environment.json`
2. You can pass it as a parameter when creating the session
3. You can set it later by `session.home = <YOUR_IRODS_PATH>`

If none of the options are used, the `session` will set it automatically to `/<zonename>/home/<username>`.

**Note**, that it is not verified that this path really exists on the iRODS server. It is merely a configuration item.

### Verify that your current working directory exists

In [ ]:
from ibridges import IrodsPath
home_path = IrodsPath(session, session.home)
print(home_path)
print(f"Path exists: {home_path.exists()}")
print(f"Path is a collection: {home_path.collection_exists()}")
print(f"Path is a data object: {home_path.dataobject_exists()}")

You can address your iRODS home in the following ways:

In [ ]:
print(IrodsPath(session))
print(IrodsPath(session, session.home))
print(IrodsPath(session, "~"))

## iRODS path manipulations

###  Create an IrodsPath

In [ ]:
irods_path = IrodsPath(session, 'my_study', 'my_experiment')

Let's inspect the properties and functions of the empty iRODS path.

In [ ]:
print(irods_path)
print(irods_path.absolute())
print(irods_path.parts)
print(irods_path.parent)
print(irods_path.exists())

When we create a new iRODS path which does not start with `/` the `IrodsPath` will automatically expand the path with the current working directory, i.e the iRODS home.
**Let's see how to omit the automatic expansion of the path with your `session.home`.**

In [ ]:
ipath = IrodsPath(session, "/", "some", "other", "irods_coll")
print(ipath)
print(ipath.absolute())
print(ipath.parts)
print(ipath.exists())

**Note,** that creating an `IrodsPath` instance does not automatically create the collection or data object on the iRODS server!

### Extending an iRODS path

Assume we would like to create a new path in iRODS e.g. to upload data to or to create a new collection. We want to extend our `home_path` with `testdata/experiment1`.

In [ ]:
new_loc = home_path.joinpath('testdata', 'experiment1')
print(new_loc)

### Joining iRODS and Windows paths
Assume you are working on windows and would like to create a copy of a local working folder on iRODS. Windows uses a different parts separator `\`. In such a case we advise you to work with the `pathlib.Path` or the `pathlib.WindowsPath`.

In [ ]:
from pathlib import PureWindowsPath
win_path = PureWindowsPath('windows','path','to', 'study', 'experiment2')
print(win_path)

We can assemble the new iRODS path `study/experiment2` like this:

In [ ]:
print(win_path.parts[-2:])
new_loc = home_path.joinpath(win_path.parts[-2], win_path.parts[-1])
print(new_loc)

Similarly we can extend a Windows path with a relative iRODS path: 

In [ ]:
print(irods_path)
new_loc = win_path.joinpath(irods_path.parts[0], irods_path.parts[1])
print(new_loc)

Other useful functions for path manipulations are:
- `IrodsPath.parent`, which will give you the direct parent of a path
- `IrodsPath.relative_to`, which computes a new path relative to another path

In [ ]:
irods_path.relative_to(IrodsPath(session, "~"))

## Functions to change collections and data object on the iRODS server

In the previous section we saw how to create iRODS paths and how to verify them. However, we have not yet changed something on the iRODS server. In this section we will go through some functionality to:

- Create a new collection from a path
- Retrieve properties of collections and data objects
- Get the data object or collection from iRODS by its path
- Rename/move data objects and collections
- Delete a data object or collection from its path

### Create a new collection

Let's have a look how to create a new subcollection in our iRODS home. We will make sure that `irods_path` does not already exist on the iRODS server.

In [ ]:
print(irods_path)
print(f"Already exists: {irods_path.exists()}")

So let us create this location:

In [ ]:
out = IrodsPath.create_collection(session, irods_path)
print(out)

The command above will create the whole subcollection tree if it does not exist yet and it will return the python object `out` which is of type `iRODSCollection`. In [02-Working-with-data.ipynb](02-Working-with-data.ipynb) we will show you what you can do with such a collection object.

### Rename or move a collection or data object

If we are not happy with the name of our new collection, we can change that:

In [ ]:
print(irods_path.exists())
new_path = irods_path.rename("my_cool_experiment")
print(irods_path.exists())

You see, that `irods_path` will stay the same, but will no longer point to an existing collection. The new location is returned as an `IrodsPath`:

In [ ]:
new_path

We can use the same function to move the collection to a different part of the iRODS tree, even if it does not exist yet:

In [ ]:
loc = new_path.rename(IrodsPath(session, "some", "coll", "my_cool_experiment"))
loc.exists()

### Delete a collection or data object

If you did not loose overview over where our new collection is, you can delete it like this:

In [ ]:
print(loc)
loc.exists()

In [ ]:
loc.remove()
print(loc.exists())

**Note**, again the instance of `IrodsPath` will continue to exist, while the location behind the path has been deleted from the iRODS instance.

### Size and checksum of the data object or collection behind a path

With `IrodsPath` we do not necessarily have to retrieve explicitly the data object or collection; `IrodsPath` contains some functions to get more information on the item behind the path.

In [ ]:
irods_path = IrodsPath(session)
print(irods_path.size)

We can also retrieve the checksum if the item behind the iRODS path is a data object `irods_path.checksum`.